# Models implementation

In [1]:
import time
import datetime
import numpy as np
import pandas as pd
from statsmodels.discrete.discrete_model import Logit
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from pprint import pprint
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [ ]:
# FIG_DIR = '/content/drive/MyDrive/DentistDataAnalysis/Experiments/figures/'
FIG_DIR = 'figures/'

## Metrics

In [2]:
def get_metrics(y_true, prediction):
    recall = round(recall_score(y_true, prediction), 4)
    accuracy = round(accuracy_score(y_true, prediction), 4)
    precision = round(precision_score(y_true, prediction), 4)
    roc_auc = round(roc_auc_score(y_true, prediction), 4) 
    f1 = round(f1_score(y_true, prediction), 4)
    conf_matrix = confusion_matrix(y_true, prediction).flatten()
    
    '''
    print('Test recall:\t', str(recall))
    print('Test accuracy:\t', str(accuracy))
    print('Test precision:\t', str(precision))
    print('Test ROC AUC:\t', str(roc_auc))
    print('Test F1 score:\t', str(f1))
    print('Test confusion matrix:\t'+ str(conf_matrix))
    '''
    return recall, accuracy, precision, roc_auc, f1, conf_matrix

## Logit

In [3]:
def perform_logit(X_train, y_train, X_test, y_test, X_cols, y_col, train_val_indices=None):
    X_train_df = pd.DataFrame(data=X_train, columns=X_cols)
    y_train_df = pd.DataFrame(data=y_train, columns=y_col)
    best_model = None
    best_score = None
    start_time = None
    end_time = None
    results = None
    if train_val_indices is not None:
        X_train_lg, X_test_lg = X_train_df.iloc[train_val_indices[0][0]], X_train_df.iloc[train_val_indices[0][1]]
        y_train_lg, y_test_lg = y_train_df.iloc[train_val_indices[0][0]], y_train_df.iloc[train_val_indices[0][1]]
        
        X_train_lg = sm.add_constant(X_train_lg)
        logit = Logit(y_train_lg, X_train_lg)
        start_time = time.time()
        results = logit.fit()
        end_time = time.time()
        X_test_lg = sm.add_constant(X_test_lg)
        yhat = results.predict(X_test_lg)
        prediction = list(map(round, yhat))
        recall = round(recall_score(y_test_lg, prediction), 4)
        best_score = recall
        best_model = results
    else:
        skf = StratifiedKFold(n_splits=5)
        start_time = time.time()
        for train_index, test_index in skf.split(X_train_df, y_train_df):
            X_train_lg, X_test_lg = X_train_df.iloc[train_index], X_train_df.iloc[test_index]
            y_train_lg, y_test_lg = y_train_df.iloc[train_index], y_train_df.iloc[test_index]

            X_train_lg = sm.add_constant(X_train_lg)
            logit = Logit(y_train_lg, X_train_lg)
            results = logit.fit()
            X_test_lg = sm.add_constant(X_test_lg)
            yhat = results.predict(X_test_lg)
            prediction = list(map(round, yhat))
            recall = round(recall_score(y_test_lg, prediction), 4)

            if best_score==None or best_score<recall:
                best_score = recall
                best_model = results
        end_time = time.time()

    conv_time = datetime.timedelta(seconds=end_time-start_time)
    
    params = results.params
    X_test = sm.add_constant(X_test)
    yhat = best_model.predict(X_test)
    prediction = list(map(round, yhat))
    return best_score, get_metrics(y_test, prediction), conv_time, params

## Decision trees

In [4]:
def perform_decision_tree(X_train, y_train, X_test, y_test, train_val_indices=None):
    #hyperparameters config
    criterion = ['gini', 'entropy']
    splitter = ['best', 'random']
    max_features = ['sqrt']
    max_depth = [int(x) for x in np.linspace(10, 50, num = 6)]
    max_depth.append(None)
    min_samples_split = [2, 5, 10]
    min_samples_leaf = [1, 2, 4]
    search_grid = {'criterion': criterion,
                   'splitter': splitter,
                   'max_features': max_features,
                   'max_depth': max_depth,
                   'min_samples_split': min_samples_split,
                   'min_samples_leaf': min_samples_leaf}
    #grid search config
    dtc_scoring='recall'
    dtc_cv = 5
    if train_val_indices is not None:
        dtc_cv = train_val_indices
    dtc_verbose=1
    dtc_n_jobs=-1
    dtc_return_train_score=True
    dtc = DecisionTreeClassifier()
    dtc_grid = GridSearchCV(estimator=dtc, 
                            param_grid=search_grid,
                            scoring=dtc_scoring, cv=dtc_cv, 
                            verbose=dtc_verbose, n_jobs=dtc_n_jobs, 
                            return_train_score=dtc_return_train_score)
    start_time = time.time()
    dtc_grid.fit(X_train, y_train)
    end_time = time.time()
    conv_time = datetime.timedelta(seconds=end_time-start_time)
    
    val_score = round(dtc_grid.best_score_, 4)
    params = dtc_grid.best_params_
    
    dtc_grid.best_estimator_.fit(X_train, y_train)
    prediction = dtc_grid.best_estimator_.predict(X_test)
    return val_score, get_metrics(y_test, prediction), conv_time, params

## Random forest

In [5]:
def perform_random_forest(X_train, y_train, X_test, y_test, title, train_val_indices=None):
    #hyperparameters config
    n_estimators = [int(x) for x in np.linspace(start = 5, stop = 150, num = 15)]
    criterion = ['gini', 'entropy']
    max_features = ['sqrt']
    max_depth = [int(x) for x in np.linspace(10, 110, num = 10)]
    max_depth.append(None)
    min_samples_split = [2, 5, 10]
    min_samples_leaf = [1, 2, 4]
    bootstrap = [True]
    random_grid = {'n_estimators': n_estimators,
                   'criterion': criterion,
                   'max_features': max_features,
                   'max_depth': max_depth,
                   'min_samples_split': min_samples_split,
                   'min_samples_leaf': min_samples_leaf,
                   'bootstrap': bootstrap}
    #grid search config
    rf_scoring='recall'
    rf_cv = 5
    if train_val_indices is not None:
        rf_cv = train_val_indices
    rf_verbose=1
    rf_n_jobs=-1
    rf_return_train_score=True
    rf = RandomForestClassifier()
    rf_grid = GridSearchCV(estimator=rf, 
                            param_grid=random_grid,
                            scoring=rf_scoring, cv=rf_cv, 
                            verbose=rf_verbose, n_jobs=rf_n_jobs, 
                            return_train_score=rf_return_train_score)
    start_time = time.time()
    rf_grid.fit(X_train, y_train)
    end_time = time.time()
    conv_time = datetime.timedelta(seconds=end_time-start_time)
    
    val_score = round(rf_grid.best_score_, 4)
    params = rf_grid.best_params_
 
    rf_grid.best_estimator_.fit(X_train, y_train)
    prediction = rf_grid.best_estimator_.predict(X_test)
    metrics = get_metrics(y_test, prediction)
    
    start_time = time.time()
    importances = rf_grid.best_estimator_.feature_importances_
    std = np.std([rf_grid.best_estimator_.feature_importances_ for tree in rf_grid.best_estimator_.estimators_], axis=0)
    elapsed_time = time.time() - start_time
    print(f"Elapsed time to compute the importances: " 
          f"{elapsed_time:.3f} seconds")
    forest_importances = pd.Series(importances, index=X_cols).sort_values(ascending=False)
    fig, ax = plt.subplots()
    forest_importances.plot.bar(yerr=std, ax=ax)
    ax.set_title("Feature importances using MDI")
    ax.set_ylabel("Mean decrease in impurity")
    ax.set_title(title)
    fig.tight_layout()
    fig.savefig(FIG_DIR+title.replace(' ', '_')+'.png', bbox_inches='tight', dpi=fig.dpi)
    return val_score, metrics, conv_time, params